In [111]:
import pandas as pd
import os
from os import walk
from collections import defaultdict
from tqdm import tqdm
from typing import List

In [112]:
DEAKIN_DATA_PATH = '/mnt/DEAKIN/DeakinImages'

In [113]:
subject_folder_paths = [os.path.join(DEAKIN_DATA_PATH, subject_id) for subject_id in os.listdir(DEAKIN_DATA_PATH)]
# filtered_image_folder_paths = [os.path.join(DEAKIN_DATA_PATH, subject_id, f'filtered_images {subject_id}') for subject_id in os.listdir(DEAKIN_DATA_PATH)]
filtered_image_folder_paths = [os.path.join(DEAKIN_DATA_PATH, subject_id, f'filtered_images') for subject_id in os.listdir(DEAKIN_DATA_PATH)]
filtered_image_folder_paths[0] = '/mnt/DEAKIN/DeakinImages/83-101/filtered_images 83-101'
filtered_image_folder_paths

['/mnt/DEAKIN/DeakinImages/83-101/filtered_images 83-101',
 '/mnt/DEAKIN/DeakinImages/83-102/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-103/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-104/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-105/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-106/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-107/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-108/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-109/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-110/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-111/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-112/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-113/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-114/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-115/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-116/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-117/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-118/filtered_images',
 '/mnt/DEAKIN/DeakinImages/83-119/filtered_images',
 '/mn

In [114]:
EXAMPLE_SUBJECT_FOLDER_PATH = filtered_image_folder_paths[1]
EXAMPLE_SUBJECT_FOLDER_PATH

'/mnt/DEAKIN/DeakinImages/83-102/filtered_images'

In [115]:
#print(os.listdir(EXAMPLE_SUBJECT_FOLDER_PATH))
EXAMPLE_CONTENT = os.listdir(os.path.join(EXAMPLE_SUBJECT_FOLDER_PATH, os.listdir(EXAMPLE_SUBJECT_FOLDER_PATH)[0]))
print(EXAMPLE_CONTENT)

['20160713_235238_000.jpg', '20160713_055250_000.jpg', '20160713_230454_000.jpg', '20000101_191411_000.jpg', '20000104_133323_000.jpg', '20160713_225612_000.jpg', '20160706_043058_000.jpg', '20160713_234354_000.jpg', '20000101_191547_000.jpg', 'prob.csv', '20000104_124449_000.jpg', '20160726_090937_000.jpg', '20000101_191723_000.jpg', '20160713_235616_000.jpg', '20160706_024618_000.jpg', '20000105_094024_000.jpg', '20160713_231338_000.jpg', '20160724_061652_000.jpg', '20160714_000327_000.jpg', '20160713_225642_000.jpg', '20160713_230758_000.jpg', '20000101_190435_000.jpg', '20160713_234659_000.jpg', '20160713_235105_000.jpg', '20160714_002602_000.jpg', '20000104_133811_000.jpg', '20160713_230931_000.jpg', '20160714_001241_000.jpg', '20160724_055311_000.jpg', '20000101_172900_000.jpg', '20160713_225542_000.jpg', '20160713_225814_000.jpg', '20160724_043319_000.jpg', '20000101_191235_000.jpg', '20000101_190051_000.jpg', '20160704_071859_000.jpg', '20000101_191059_000.jpg', '20160713_23534

In [116]:
EXAMPLE_LABEL = [os.listdir(EXAMPLE_SUBJECT_FOLDER_PATH)[0]] * len(EXAMPLE_CONTENT)

In [117]:
df = pd.Series(index=EXAMPLE_CONTENT, data=EXAMPLE_LABEL)
df.head()

20160713_235238_000.jpg    ScreenA
20160713_055250_000.jpg    ScreenA
20160713_230454_000.jpg    ScreenA
20000101_191411_000.jpg    ScreenA
20000104_133323_000.jpg    ScreenA
dtype: object

In [118]:
def convert_to_table_format(subject_annotations: defaultdict(list)) -> List[List[str]]:
    max_num_labels = max([len(labels) for _, labels in annotations.items()])
#     print("Maximum number of labels: ", max_num_labels)
    # Convert to table-type format
    rows = []
    for image_id, labels in annotations.items():
        single_row_data = [image_id, *labels]
        if len(single_row_data) < max_num_labels + 1:
            num_remaining_items = max_num_labels + 1 - len(single_row_data)
            single_row_data += [None] * num_remaining_items
        rows.append(single_row_data)
    return rows

In [119]:
for subject in tqdm(filtered_image_folder_paths):
    subject_id = subject.split('/')[-2]
    annotations = defaultdict(list)
    for label in os.listdir(subject):
        if label == '.DS_Store': 
            continue
        subject_label_path = os.path.join(subject, label)
        image_ids = sorted([image_id for image_id in os.listdir(subject_label_path) if os.path.splitext(image_id)[-1] != '.csv'])
        labels_for_image_ids = [label] * len(image_ids)
        for index, image_id in enumerate(image_ids):
            corresponding_label = labels_for_image_ids[index] # Get the corresponding label for each image_id
            annotations[image_id].append(corresponding_label) # Add it to the corresponding entry in the dictionary
        # Convert annotation dictionary data structure into table format structure
        max_num_labels = max([len(labels) for _, labels in annotations.items()])
        rows = convert_to_table_format(annotations)
        column_names = ['image_id', *[f'label_{index + 1}' for index in range(max_num_labels)]]
        df = pd.DataFrame(rows, columns=column_names)
        df.to_csv(f'prev_annotations/{subject_id}_prev_annotations.csv', index=False)
# print(annotations)

100%|████████████████████████████████████████████████████████████████| 30/30 [00:57<00:00,  1.92s/it]


Maximum number of labels:  3


100%|█████████████████████████████████████████████████████| 23850/23850 [00:00<00:00, 1302290.60it/s]


,image_id,label_1,label_2,label_3
0,20000101_135119_000.jpg,ScreenA,None,None
1,20000101_150240_000.jpg,ScreenA,None,None
2,20000101_155600_000.jpg,ScreenA,None,None
3,20000101_165738_000.jpg,ScreenA,None,None
4,20000102_121847_000.jpg,ScreenA,None,None


In [121]:
name = '20000101_135119_000.jpg'
print(os.path.splitext(name)[-2])

20000101_135119_000
